In [3]:
# Import and configure the sentiment analyzer.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

TWEETS_DATABASE_FILENAME = 'tweets_data.db'

In [8]:
conn = sqlite3.connect(TWEETS_DATABASE_FILENAME)

with conn:
    cur = conn.cursor()

    print('Getting tweets...')
    completed_tweets_count = cur.execute("""
        SELECT
            COUNT(id)
        FROM
            tweets
        WHERE
            sentiment IS NOT NULL
    """).fetchone()[0]
    
    print(completed_tweets_count, 'tweets already have sentiment data')


Getting tweets...
314104 tweets already have sentiment data


In [ ]:
import sqlite3
import os
import json

conn = sqlite3.connect(TWEETS_DATABASE_FILENAME)
conn.isolation_level = "DEFERRED"

has_null_sentiment_tweets = True

completed_batches_count = 0

insert_batch_size = 10000


def insert_batch_to_db(values_to_commit):
    """
    Batch insert tweet data into the database.
    """
    sql = """
        INSERT OR REPLACE INTO
            tweets (id, sentiment, data) 
        VALUES (
            ?, 
            ?,
            (
                SELECT
                    data
                FROM
                    tweets
                WHERE
                    id = ?
            )
        );
    """
    cur.executemany(sql, values_to_commit)


def generate_and_insert_tweets(results):
    """
    Generates sentiment data and updates the database with the new data, given SQL results.
    """
    values_to_commit = []
    for index, result in enumerate(results):
        tweet_text = result[0]
        tweet_id = result[1]

        sentiment_scores = analyser.polarity_scores(tweet_text)

        values_to_commit.append(
            [tweet_id, json.dumps(sentiment_scores), tweet_id])

        if (index + 1) % insert_batch_size == 0:
            insert_batch_to_db(values_to_commit)

            print('updated tweet',
                  (completed_batches_count * 50000) + index + 1)
            values_to_commit = []  # Reset to empty


with conn:
    cur = conn.cursor()

    while has_null_sentiment_tweets:
        print('Getting tweets...')

        # Query sentiment data in batches of 50k
        cur.execute("""
            SELECT
                json_extract(data, '$.text'), id
            FROM
                tweets
            WHERE
                sentiment IS NULL
            LIMIT 50000
        """)
        results = cur.fetchall()

        # Case: all tweets already have sentiment data
        if len(results) == 0:
            has_null_sentiment_tweets = False
            break

        values_to_commit = []

        cur.execute("begin")
        try:
            generate_and_insert_tweets(results)
        except sqlite3.Error as e:
            cur.execute('rollback')
        finally:
            conn.execute('commit')

        completed_batches_count += 1

Getting tweets...


In [12]:
import sqlite3

conn = sqlite3.connect(TWEETS_DATABASE_FILENAME)

with conn:
    cur = conn.cursor()
    print('Getting count...')
    cur.execute("SELECT COUNT(id) FROM tweets WHERE sentiment NOT NULL")
    print(cur.fetchone()[0], 'database record(s) have sentiment information')

Getting count...
629111 database record(s) have sentiment information
